# Functions from Stream to Stream

This module describes one way of creating functions from a single stream to a single stream. Other ways of mapping a single input stream to a single output stream are described elsewhere.

# Mapping a single input element to a single output element
## @fmap_e

We use the decorator @fmap_e to convert a function that maps an element to an element into a function that maps a stream into a stream.

@fmap_e
def f(v): return v+10

Creates a function f from a stream to a stream where the n-th element of the output stream is (undecorated) f applied to the n-th element of the input stream.

If y = f(x) then y[n] = x + 10

In [1]:
import os
import sys
sys.path.append("../")

from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.op import map_element
from IoTPy.agent_types.basics import fmap_e
from IoTPy.helper_functions.recent_values import recent_values

@fmap_e
def f(v): return v+10

# f is a function that maps a stream to a stream

def example():
    x = Stream()
    y = f(x)
    
    x.extend([1, 2, 3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[11, 12, 13, 14]


# map_element

In some situations, you may not want to decorate a function. In these cases you can use *map_element* 

*map_element(func, in_stream, out_stream, state=None, name=None, **kwargs)*

where *func* is the function applied to each element of the input stream. 

Next, we implement the previous example without using decorators. Note that you have to declare *x* **AND** *y* as streams, and specify the relation between *x* and *y* by calling *map_element* **before** extending stream *x*.


In [26]:
def example():
    def f(v): return v+10
    
    x, y = Stream(), Stream()
    map_element(func=f, in_stream=x, out_stream=y)
    
    x.extend([1, 2, 3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[11, 12, 13, 14]


## Mapping element to element : function with keyword arguments
The function 

*f(v, addend) = return v * multiplier + addend* 

maps *v* to the return value. The first parameter *v* is an element of an input stream, and the arguments *addend* and *multiplier* are keyword arguments of the function.

Decorating the function with @fmap_e gives a function that maps a stream to a stream. 

In [2]:
@fmap_e
def f(v, addend, multiplier): return v * multiplier + addend

# f is a function that maps a stream to a stream

def example():
    x = Stream()
    # Specify the keyword argument: addend
    y = f(x, addend=20, multiplier=2)
    
    x.extend([1, 2, 3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[22, 24, 26, 28]


## Mapping element to element: Function with state

Strictly speaking a function cannot have state; however, we bend the definition here to allow functions with states that operate on streams.

Look at this example: The input and output streams of a function are x and y, respectively; and we want:

y[n] = (x[0] + x[1] + ... + x[n]) * multiplier

where multiplier is an argument.

We can implement a function where its state before the n-th application of the function is:

x[0] + x[1] + ... + x[n-1], 

and its state after the n-th application is:

x[0] + x[1] + ... + x[n-1] + x[n]

The state is updated by adding x[n].

We can capture the *state* of a function by specifying a special keyword argument **state** and specifying the initial state in the call to the function. The function must returns two values: the next output and the next state.

In this example, the function has 3 values: the next element of the stream, state, and multiplier. The state and multiplier are keyword arguments.


In [3]:
@fmap_e
def f(v, state, multiplier):
    output = (v + state)*multiplier
    next_state = v + state
    return output, next_state

# f is a function that maps a stream to a stream

def example():
    x = Stream()
    # Specify the keyword argument: multiplier
    y = f(x, state=0, multiplier=2)
    
    x.extend([1, 2, 3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[2, 6, 12, 20]


## Same example using map_element instead of a decorator

In [29]:
def f(v, state, multiplier):
    output = (v + state)*multiplier
    next_state = v + state
    return output, next_state


def example():
    x, y = Stream(), Stream()

    map_element(func=f, in_stream=x, out_stream=y, state=0, multiplier=2)
    
    x.extend([1, 2, 3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[2, 6, 12, 20]


## Saving state in an object

You can save the state of a stream in an object such as a **dict** as shown in the following example of the Fibonacci sequence.

In [4]:
def example():
    x = Stream('x')
    
    # Object in which state is saved
    s = {'a':0, 'b':1}
    
    @fmap_e
    def fib(v, fib):
        # Update state
        fib['a'], fib['b'] = fib['a'] + fib['b'], fib['a']
        return fib['a'] + v

    # Declare stream y
    y = fib(x, fib=s)
    
    
    x.extend([0, 0, 0, 0, 0, 0, 0])

    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[1, 1, 2, 3, 5, 8, 13]


## Filtering elements in a stream

We are given a function *f* that returns a Boolean. We apply the decorator @filter_e to get a function that takes an input stream and returns an output stream consisting of those elements in the input stream for which *f* returns **True**.

In the following example, *positive(v)* returns **True** exactly when *v* is positive. After we apply the decorator, *f* becomes a function that reads an input stream and returns a stream consisting of the input stream's positive values.


In [5]:
from IoTPy.agent_types.basics import filter_e

@filter_e
def positive(v, threshold): return v > threshold

# f is a function that maps a stream to a stream

def example():
    x = Stream()
    y = positive(x, threshold=0)
    
    x.extend([-1, 2, -3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[2, 4]


## Using filter_element instead of a decorator

Just as you may prefer to use map_element instead of the decorator fmap_e in some situations, you may also prefer to use filter_element instead of the decorator filter_e. The previous example, implemented without decorators, is given next.

In [34]:
from IoTPy.agent_types.op import filter_element

def example():
    def positive(v, threshold): return v > threshold
    
    x, y = Stream(), Stream()
    filter_element(func=positive, in_stream=x, out_stream=y, threshold=0)
    
    x.extend([-1, 2, -3, 4])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[2, 4]


## Function that maps list to list

In some cases, using functions that map lists to lists is more convenient than functions that map element to element.
When such a function is decorated with @fmap_l, the function becomes one that maps streams to streams.

Example: Decorate the function *increment_odd_numbers*

In [6]:
from IoTPy.agent_types.basics import fmap_l

In [7]:
@fmap_l
def increment_odd_numbers(the_list):
    return [v+1 if v%2 else v for v in the_list]

def example():
    x = Stream()
    y = increment_odd_numbers(x)
    
    x.extend([0, 1, 2, 3, 4, 5, 6])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[0, 2, 2, 4, 4, 6, 6]


## Example: incremental computations from list to list

Given a list *x* we can generate list *y* where *y[j] = x[0] + .. + x[j]* by:

* y = list(accumulate(x))*

For example, if *x = [1, 2, 3]* then *y = [1, 3, 6]*

Now suppose we extend *x* with the list *[4, 5, 6, 7]*, then we can get the desired *y = [1, 3, 6, 10, 15, 21, 28]* by calling *accumulate* again. We can also compute the new value of *y* **incrementally** by adding the last output from the previous computation (i.e., 6) to the accumulation of the extension, as shown next.

In [8]:
from itertools import accumulate

def incremental_accumulate(the_list, state):
    print ("the_list ", the_list)
    output_list = [v + state for v in list(accumulate(the_list))]
    next_state = output_list[-1]
    return output_list, next_state

def example():
    x = [1, 2, 3]
    y, state = incremental_accumulate(x, state=0)
    print ('y is ', y)
    x.extend([4, 5, 6, 7])
    y, state = incremental_accumulate(x, state=0)
    print ('y is ', y)
    
example()


the_list  [1, 2, 3]
y is  [1, 3, 6]
the_list  [1, 2, 3, 4, 5, 6, 7]
y is  [1, 3, 6, 10, 15, 21, 28]


## Incremental computations from stream to stream

We can decorate the incremental computation from list to list to obtain a computation from stream to stream. This is illustrated in the next example.

In [10]:
from itertools import accumulate

@fmap_l
def incremental_accumulate(the_list, state):
    output_list = [v + state for v in list(accumulate(the_list))]
    next_state = output_list[-1]
    return output_list, next_state

def example():
    x = Stream()
    y = incremental_accumulate(x, state=0)
    
    x.extend([10, 20, -30, 50, -40])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))
    
    x.extend([10, 20, -30])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[10, 30, 0, 50, 10]
recent values of stream y are
[10, 30, 0, 50, 10, 20, 40, 10]


## Example with state and keyword argument

We want to output the elements of the accumulated stream that exceed a threshold. For example, if a stream x is [10, 20, -30, 50, -40] then the accumulation stream is [10, 30, 0, 50, 10] and the elements of the accumulated stream that exceed a threshold of 25 are [30, 50].

In [11]:
from itertools import accumulate

@fmap_l
def total_exceeds_threshold(the_list, state, threshold):
    output_list = [v + state for v in list(accumulate(the_list)) if v + state > threshold]
    state += sum(the_list)
    return output_list, state

def example():
    x = Stream()
    y = total_exceeds_threshold(x, state=0, threshold=25)
    
    x.extend([10, 20, -30, 50, -40])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))
    
    x.extend([10, 20, -30])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[30, 50]
recent values of stream y are
[30, 50, 40]


## Example: function composition

We can also solve the previous problem by concatenating the functions *positive* and *incremental_accumulate*.

In [25]:
def example():
    x = Stream()
    y = positive(incremental_accumulate(x, state=0), threshold=25)
    x.extend([10, 20, -30, 50, -40])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))
    
    x.extend([10, 20, -30])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))


example()

recent values of stream y are
[30, 50]
recent values of stream y are
[30, 50, 40]


## Using list_element instead of the decorator fmap_l

The next example illustrates how map_element can be used with state and keyword arguments. It is the same as the previous example, except that it doesn't use decorators.

In [37]:
from IoTPy.agent_types.basics import map_list

def total_exceeds_threshold(the_list, state, threshold):
    output_list = [v + state for v in list(accumulate(the_list)) if v + state > threshold]
    state += sum(the_list)
    return output_list, state

def example():
    x, y = Stream(), Stream()
    map_list(func=total_exceeds_threshold, in_stream=x, out_stream=y, state=0, threshold=25)
    
    x.extend([10, 20, -30, 50, -40])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))
    
    x.extend([10, 20, -30])
    
    # Execute a step
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example()

recent values of stream y are
[30, 50]
recent values of stream y are
[30, 50, 40]
